In [2]:
import time
import numpy as np
import uuid
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options

# Set Headers and Context Options
headers = {'Accept-Encoding': 'gzip, deflate, sdch',
           'Accept-Language': 'en-US,en;q=0.8',
           'Upgrade-Insecure-Requests': '1',
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
           'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
           'Cache-Control': 'max-age=0',
           'Connection': 'keep-alive'}

C:\Users\ASUS\AppData\Local\Temp\ipykernel_53568\865737341.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# Define firefox setting
firefox_options = Options()
for key, value in headers.items():
    firefox_options.set_preference("network.http.header.override." + key, value)
driver = webdriver.Firefox(options = firefox_options)

In [4]:
driver.get('https://www.tokopedia.com/samsung/product?sort=11')

In [5]:
# You can set your own pause time. My laptop is a bit slow so I use 1 sec
scroll_pause_time = 1
screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
i = 1

# See maximum limit scrolling to 5
while True:
    # scroll one screen height each time
    executed = False
    while executed == False:
        try:
            driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
            i += 1.0
            time.sleep(scroll_pause_time)
            # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
            scroll_height = driver.execute_script("return document.body.scrollHeight;")  
            executed = True
        except:
            time.sleep(0.5)
    # Break the loop when the height we need to scroll to is larger than the total scroll height
    if (screen_height) * i > scroll_height:
        break

In [6]:
video_post_link = driver.find_elements(by = By.XPATH, value = "/html/body/div[1]/div/div[2]/div[2]/div[4]/div/div[2]/div[1]/div/div/div/div/div/div/div[1]/a")

In [7]:
video_post_link_copy = [html.get_attribute('href') for html in video_post_link]

In [8]:
video_post_link_copy

['https://www.tokopedia.com/samsung/samsung-galaxy-buds2-onyx?extParam=src%3Dshop%26whid%3D11442364',
 'https://www.tokopedia.com/samsung/samsung-galaxy-a24-lte-8-128gb-silver-19010?extParam=src%3Dshop%26whid%3D11442364',
 'https://www.tokopedia.com/samsung/samsung-galaxy-buds2-pro-white?extParam=src%3Dshop%26whid%3D11442364',
 'https://www.tokopedia.com/samsung/samsung-galaxy-a05s-6-128gb-black-93f2d?extParam=whid%3D11442364%26src%3Dshop',
 'https://www.tokopedia.com/samsung/samsung-galaxy-watch4-fresh-40mm-gold?extParam=src%3Dshop%26whid%3D13056835',
 'https://www.tokopedia.com/samsung/samsung-galaxy-a05-6-128gb-silver-61ad5?extParam=src%3Dshop%26whid%3D11442364',
 'https://www.tokopedia.com/samsung/samsung-galaxy-a04s-4gb-128gb-black-3f26c?extParam=src%3Dshop%26whid%3D11442364',
 'https://www.tokopedia.com/samsung/samsung-galaxy-s23-ultra-5g-12-256gb-lavender-8b75?extParam=src%3Dshop%26whid%3D11442364',
 'https://www.tokopedia.com/samsung/samsung-galaxy-m34-5g-8-128gb-blue-c3e4b?ext

In [11]:
def scroll_website(max_scroll:int=5):
    # You can set your own pause time. My laptop is a bit slow so I use 1 sec
    scroll_pause_time = 1
    screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
    i = 1.0

    # See maximum limit scrolling to 5
    while True:
        # scroll one screen height each time
        executed = False
        while executed == False:
            try:
                driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
                i += 1.0
                time.sleep(scroll_pause_time)
                # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
                scroll_height = driver.execute_script("return document.body.scrollHeight;")  
                executed = True
            except:
                time.sleep(0.5)
        # Break the loop when the height we need to scroll to is larger than the total scroll height
        if (screen_height) * i > scroll_height or i > max_scroll:
            break

In [19]:
def scrape_items(url):
    # Get URL
    driver.get(url)

    time.sleep(3)

    # Scroll web
    scroll_website(10)

    # Get Item Name
    item_name = driver.find_element(by = By.XPATH, value = '/html/body/div[1]/div/div[2]/div[2]/div[4]/div/h1').text
    sold_count = driver.find_element(by = By.XPATH, value = '/html/body/div[1]/div/div[2]/div[2]/div[4]/div/div[1]/div/p[1]').text
    rating = driver.find_element(by = By.XPATH, value = '/html/body/div[1]/div/div[2]/div[2]/div[4]/div/div[1]/div/p[2]/span[1]/span[2]').text

    # Scroll web
    description = ''
    try:
        button_readmore = driver.find_element(by = By.XPATH, value = '/html/body/div[1]/div/div[2]/div[2]/div/div[2]/div[2]/div/button')
        button_readmore.click()
        description += driver.find_element(by = By.XPATH, value = '/html/body/div[1]/div/div[2]/div[2]/div/div[2]/div[2]/ul').get_attribute('innerText') + '\n\nDETAIL:\n'
        description += driver.find_element(by = By.XPATH, value = '/html/body/div[1]/div/div[2]/div[2]/div/div[2]/div[2]/div/span/span/div').get_attribute('innerText')
    except:
        description += driver.find_element(by = By.XPATH, value = '/html/body/div[1]/div/div[2]/div[2]/div[6]/div[2]/div[2]/ul').get_attribute('innerText')
        description += driver.find_element(by = By.XPATH, value = '/html/body/div[1]/div/div[2]/div[2]/div/div[2]/div[2]/div/span/span/div').get_attribute('innerText')

    driver.execute_script("window.scrollTo(0, 200);")
    specification = ''
    try:
        more_info_button = driver.find_element(by = By.XPATH, value = '/html/body/div[1]/div/div[2]/div[2]/div/div[2]/div[1]/div/button[3]')
        more_info_button.click()
        specification_list = driver.find_elements(by = By.XPATH, value = '/html/body/div[1]/div/div[2]/div[2]/div/div[2]/div[2]/ul/li/button')
        if len(specification_list) > 0:
            for button in specification_list:
                button.click()
                time.sleep(2)
                specification += driver.find_element(by = By.XPATH, value = '//*[@id="tab-note-panel"]').get_attribute('innerText') + '\n'
                specification += driver.find_element(by = By.XPATH, value = '/html/body/div/div[2]/article/div/div/div/div[2]/div[2]').get_attribute('innerText')
                close_button = driver.find_element(by = By.XPATH, value = '/html/body/div/div[2]/article/header/button')
                close_button.click()
                time.sleep(2)
    except Exception as e:
        print(e)

    # Get total stocks
    total_stocks = driver.find_element(by = By.XPATH, value = '/html/body/div[1]/div/div[2]/div[2]/div[3]/div/div/div[1]/p').get_attribute('innerText')
    id = uuid.uuid4()

    dict_item = {
        'pid': id,
        'URL': url,
        'Item Name': item_name,
        'Description': description,
        'Specification': specification,
        'Total Stock': total_stocks,
        'Quantity Sold': sold_count,
        'Rating': rating
    }

    max_qna_page = 3

    item_id_list = []
    question_list = []
    user_q_list = []
    answer_list = []
    user_a_list = []

    for i in range(max_qna_page):
        # Get QnA box
        qna_box = driver.find_element(by = By.XPATH, value = '//*[@id="pdp_comp-discussion_faq"]/div')

        # Get questions
        article_list = qna_box.find_elements(by = By.XPATH, value = 'div/section/article')

        # Find expand buttons
        button_lists = qna_box.find_elements(by = By.XPATH, value = 'div/section/article/div[2]/div/ul/button')
        for button in button_lists:
            try:
                button.click()
            except Exception as e:
                continue
        for i, article in enumerate(article_list):
            question = article.find_element(by = By.XPATH, value = 'div[1]/div/div[1]/div/p').text
            user_q = article.find_element(by = By.XPATH, value = 'div[1]/div/div[1]/div/div/a').text
            try:
                reply_box = article.find_element(by = By.XPATH, value = 'div[2]/div/ul/li[1]/div/div[1]/div')
                if article.find_element(by = By.XPATH, value = 'div[2]/div/ul/li[1]/div/div[1]/div/div/div').text == 'Penjual':
                    answer = reply_box.find_element(by = By.XPATH, value = 'p/span').text
                    user_a = reply_box.find_element(by = By.XPATH, value = 'div/a').text
            except Exception as e:
                continue
            item_id_list.append(id)
            question_list.append(question)
            user_q_list.append(user_q)
            answer_list.append(answer)
            user_a_list.append(user_a)
        
        # Click next button
        # /html/body/div[1]/div/div[2]/div[2]/div[15]/div/div/section/div/nav/ul/li[10]/button
        driver.find_elements(by = By.XPATH, value = "/html/body/div[1]/div/div[2]/div[2]/div[15]/div/div/section/div/nav/ul/li/button")[-1].click()

        time.sleep(2)
    
    # Create dict_qna
    dict_qna = {
        'Item ID': item_id_list,
        'User Question': user_q_list,
        'Question': question_list,
        'User Answer': user_a_list,
        'Answer': answer_list}
    

    return dict_item, dict_qna

In [20]:
item_list = []
qna_df = pd.DataFrame()
for url in video_post_link_copy[:5]:
    dict_item, dict_qna = scrape_items(url)
    item_list.append(dict_item)
    qna_df = pd.concat([
        qna_df,
        pd.DataFrame(dict_qna)
    ], axis = 0, ignore_index = True)

In [21]:
qna_df

,Item ID,User Question,Question,User Answer,Answer
0,28861625-80f9-46a5-abbb-8a890d12de2b,Satrio,Ini support wireless charging ga?,Samsung Official Store,"Hi kak, untuk unit support wireless charging, ..."
1,28861625-80f9-46a5-abbb-8a890d12de2b,Tasya,onyx ready? bs utk s22 dan flip3 ?,Samsung Official Store,"Hi kak, untuk unit support pada OneUI 3.1 dan ..."
2,28861625-80f9-46a5-abbb-8a890d12de2b,Natasha,Bisa utk A34 5G?,Samsung Official Store,"Hi kak, untuk unit kompetibel ya kak, Terima k..."
3,28861625-80f9-46a5-abbb-8a890d12de2b,safril,kak kompatibel sama galaxy a12 ga ya?,Samsung Official Store,"Hi kak, untuk unit Galaxy Buds2 mendukung pada..."
4,28861625-80f9-46a5-abbb-8a890d12de2b,Muhammad,Black ready ?,Samsung Official Store,Hi kak! Untuk saat ini stok masih tersedia. Si...
...,...,...,...,...,...
139,0da69cfd-108d-45b6-b9e3-c6cd06261eae,Betty,untuk link chargernya yg mana ya kak? apakah a...,Samsung Official Store,Hi kak! Mohon maaf untuk saat ini masih belum ...
140,0da69cfd-108d-45b6-b9e3-c6cd06261eae,Adam,untuk charging doc nya juga gadapet gan ?,Samsung Official Store,Hi Kak. Mohon maaf belum dengan charger ya kak...
141,0da69cfd-108d-45b6-b9e3-c6cd06261eae,cadgtop,"ini dah ada bluetooth, wifi dan gps ya?",Samsung Official Store,Hi Kak. Untuk unit tersebut menggunakan blueto...
142,0da69cfd-108d-45b6-b9e3-c6cd06261eae,Dhani,bisa sameday gak kak?,Samsung Official Store,Hi Kak. Mohon maaf atas ketdiaknyamananya untu...


In [56]:
item_df = pd.DataFrame(item_list)

In [57]:
item_df.to_csv('../data/item_metadata.csv', index = False)

In [59]:
qna_df.to_csv('../data/qna_metadata.csv', index = False)